In [9]:
import copy
import json
import math
import os
import pathlib
import time
import psycopg2
import requests
from requests.adapters import HTTPAdapter
from requests.auth import HTTPBasicAuth
from urllib3.util.retry import Retry

In [10]:
PL_API_KEY = 'PLAK721cf7576b5f4835bcd0f2dfe9a7a395'
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

SESSION = requests.Session()

SESSION.auth = (PL_API_KEY, '')
AUTH = HTTPBasicAuth(PL_API_KEY, '')

retries = Retry(total=10, backoff_factor=1, status_forcelist=[429])
SESSION.mount('https://', HTTPAdapter(max_retries=retries))

In [11]:
class PGConn:
    def __init__(self, host, port, dbname, user=None, passwd=None):
        self.host = host
        self.port = port
        self.dbname = dbname
        if user is not None:
            self.user = user
        else:
            self.user = ""
        if passwd is not None:
            self.passwd = passwd
        else:
            self.passwd = ""
        self.conn = None

    def connection(self):
        """Return connection to PostgreSQL.  It does not need to be closed
        explicitly.  See the destructor definition below.

        """
        if self.conn is None:
            conn = psycopg2.connect(dbname=self.dbname,
                                    host=self.host,
                                    port=str(self.port),
                                    user=self.user,
                                    password=self.passwd)
            self.conn = conn
            
        return self.conn

    def __del__(self):
        """No need to explicitly close the connection.  It will be closed when
        the PGConn object is garbage collected by Python runtime.

        """
        print(self.conn)
        self.conn.close()
        self.conn = None

pgconn_obj = PGConn(
    "localhost",
    5432,
    "dolr",
    "sameer",
    "swimgood"
)
    
pgconn=pgconn_obj.connection()

sql_query = f"""
    select
        st_astext(st_transform(geom, 3857)) as geom_text
    from
        pilot.dagdagad_farmplots_dedup
    where
        gid = 5
    ;
"""

with pgconn.cursor() as curs:
    curs.execute(sql_query)
    poly_fetch = curs.fetchone()[0]

poly_split = poly_fetch.split('(')
poly_coords = poly_split[3]
poly_coords = poly_coords.split(')')[0]
poly_point_coords = poly_coords.split(',')

points = []
for pt_coords in poly_point_coords:
    pt_split = pt_coords.split(' ')
    lat = float(pt_split[0])
    long = float(pt_split[1])
    points.append([lat, long])
print(points)

pgconn_obj.__del__()

Exception ignored in: <function PGConn.__del__ at 0x7f0ce9222440>
Traceback (most recent call last):
  File "/tmp/ipykernel_8176/2076760015.py", line 37, in __del__
AttributeError: 'NoneType' object has no attribute 'close'


None
[[8641083.055386605, 2395102.6341904276], [8641079.081280783, 2395056.3967877664], [8641076.932814611, 2395055.3592203986], [8641071.61174295, 2395057.601320072], [8641009.718106069, 2395069.396200143], [8640965.891622545, 2395073.343730003], [8640928.399218045, 2395071.8171926714], [8640885.474422395, 2395064.9835545085], [8640870.468555037, 2395064.1606558487], [8640873.407389594, 2395112.556702624], [8641000.723491214, 2395108.299085534], [8641083.055386605, 2395102.6341904276]]
<connection object at 0x7f0cd258fc40; dsn: 'user=sameer password=xxx dbname=dolr host=localhost port=5432', closed: 0>


In [12]:
def place_monthly_order(mosaic_name, points):
    order_params = {
        "name": "Basemap order with geometry",
        "source_type": "basemaps",
        "products": [
            {
                "mosaic_name": mosaic_name,
                "geometry":{
                "type": "Polygon",
                "coordinates":[
                    points
                ]
                }
            }
        ]
    }
    
    paramRes = requests.post(ORDERS_API_URL,
        data=json.dumps(order_params),
        auth=AUTH,
        headers={'content-type': 'application/json'}
    )

    print(paramRes.text)

    order_id = paramRes.json()['id']
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

def poll_for_success(order_url, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=AUTH)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)

def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))


In [13]:
# order_urls = []
# for month in range(1, 13):
#     if month < 10:
#         month = "0" + str(month)
#     else:
#         month = str(month)
#     mosaic_name = f"global_monthly_2023_{month}_mosaic"
#     order_url = place_monthly_order(mosaic_name, points)
#     order_urls.append(order_url)
    

In [14]:
# for order_url in order_urls:
#     poll_for_success(order_url)
#     r = requests.get(order_url, auth=AUTH)
#     response = r.json()
#     results = response['_links']['results']
#     print([r['name'] for r in results])
#     download_results(results)

In [15]:
import rasterio
from shapely.wkt import loads
from rasterio.mask import mask

def clip_raster_with_multipolygon(raster_path, multipolygon, output_path):
    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Clip the raster with the multipolygon
        out_image, out_transform = mask(src, [multipolygon], crop=True)
        
        # Copy the metadata
        out_meta = src.meta.copy()

        # Update the metadata to match the clipped raster
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        # Write the clipped raster to a new file
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

raster_path = 'data/0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_quad.tif'
output_path = 'temp_clipped.tif'

multipolygon = loads(poly_fetch)
print(multipolygon)

clip_raster_with_multipolygon(raster_path, multipolygon, output_path)

MULTIPOLYGON (((8641083.055386605 2395102.6341904276, 8641079.081280783 2395056.3967877664, 8641076.932814611 2395055.3592203986, 8641071.61174295 2395057.601320072, 8641009.718106069 2395069.396200143, 8640965.891622545 2395073.343730003, 8640928.399218045 2395071.8171926714, 8640885.474422395 2395064.9835545085, 8640870.468555037 2395064.1606558487, 8640873.407389594 2395112.556702624, 8641000.723491214 2395108.299085534, 8641083.055386605 2395102.6341904276)))


In [16]:
import rasterio
import numpy as np

def calculate_average_color(tif_path):
    with rasterio.open(tif_path) as src:
        # Read the raster data
        data = src.read()

        # Assuming a 3-band image (RGB), calculate the average for each band
        if data.shape[0] >= 3:
            red_avg = np.mean(data[0])
            green_avg = np.mean(data[1])
            blue_avg = np.mean(data[2])
            return (red_avg, green_avg, blue_avg)
        else:
            raise ValueError("The image is not a 3-band (RGB) image.")

tif_path = 'test_clipped.tif'
average_color = calculate_average_color(tif_path)
print(f"Average color (RGB): {average_color}")


Average color (RGB): (57.74414715719063, 42.90969899665552, 28.66722408026756)
